# Data Loaded & Preprossessed

In [2]:
#Import packages
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Load the datasets
left_df = pd.read_csv('left_dataset.csv')
right_df = pd.read_csv('right_dataset.csv')

#Preprocessing function for the names and addresses
def preprocess(text):
    #Simple case normalization and stripping leading/trailing whitespace
    return text.lower().strip()

#Apply the preprocessing function to the name and address columns
left_df['name'] = left_df['name'].astype(str).apply(preprocess)
left_df['address'] = left_df['address'].astype(str).apply(preprocess)
right_df['name'] = right_df['name'].astype(str).apply(preprocess)
right_df['address'] = right_df['address'].astype(str).apply(preprocess)

# Matching

In [3]:
#Combine name and address for a complete profile
left_df['profile'] = left_df['name'] + ' ' + left_df['address']
right_df['profile'] = right_df['name'] + ' ' + right_df['address']

#Vectorize the profiles using TF-IDF
vectorizer = TfidfVectorizer()
left_tfidf = vectorizer.fit_transform(left_df['profile'])
right_tfidf = vectorizer.transform(right_df['profile'])

#Calculate cosine similarity between all profiles
similarity_matrix = cosine_similarity(left_tfidf, right_tfidf)

#Consider a pair a match if the cosine similarity is above a threshold (e.g., 0.8)
matches = []
threshold = 0.8
for i, row in enumerate(similarity_matrix):
    for j, score in enumerate(row):
        if score > threshold:
            matches.append((left_df.iloc[i]['entity_id'], right_df.iloc[j]['business_id'], score))

#Convert matches to DataFrame
matches_df = pd.DataFrame(matches, columns=['left_entity_id', 'right_business_id', 'confidence_score'])

#Sort by confidence score in descending order
matches_df = matches_df.sort_values(by='confidence_score', ascending=False)

# Output Matches w/ High Degree of Confidence

In [7]:
print(matches_df)

      left_entity_id  right_business_id  confidence_score
1504           19399              79399          1.000000
6552           84174              76518          1.000000
5779           73679              53458          1.000000
5764           73494              71258          1.000000
5304           67914              53103          1.000000
...              ...                ...               ...
3846           49394              61960          0.800188
3845           49394              54579          0.800188
15               259              78894          0.800133
4732           60397              40404          0.800095
4731           60397              40403          0.800095

[7404 rows x 3 columns]


In [5]:
#Save the matches to a CSV file
matches_df.to_csv('/Users/sterben/Library/Mobile Documents/com~apple~CloudDocs/Documents/apan5210/group-project-spring-2024/results1.csv', index=False)